In [1]:
import tracemalloc
import globalVars
from ranker import *
from clustering import *
from searchIndexer import *
from utilities import *
tracemalloc.start()
print("Start Running: ",tracemalloc.get_traced_memory())

print("loading data")
listOfQueries = getListQueries()

##########load schema###########

tracemalloc.stop()

tracemalloc.start()
###########one hot encoding############
globalVars.init()
OneHotVocab = globalVars.OneHotVocab 
init_one_hot_vocab(OneHotVocab)


Start Running:  (471, 10620)
loading data
loading data done


In [ ]:
with open('/home/nada/GP/GP/GP/src/SearchEngine/nGrams/unigram.pickle', 'rb') as handle:
    ngrams = pickle.load(handle)

In [2]:

attr = "course.name"
if attr.find(".") != -1:
    attr = attr.split(".")[1]
attr = attr.split("_")
attr = [get_lemma(word) for word in attr]
attrOneHotVector = (getKeyWordsVector(attr)).tostring()   
print("attrOneHotVector: ",attrOneHotVector)
#ngrams["whereAtrrsDict"].get(attrOneHotVector)


attrOneHotVector:  b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x

/home/nada/.local/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  


In [ ]:
entity="players_teams"
entity = entity.split("_")
entity = [get_lemma(word) for word in entity]
print(entity)
vv= getKeyWordsVector(entity)
for v in vv:
    print(v)
entityOneHotVector = vv.tostring()
entitiesOneHotVector = [entityOneHotVector]
print(ngrams["whereAtrrsDict"].get(entityOneHotVector))
best_combination = None
best_combination_key = None
len_best_combination =0
for r in range(len(entitiesOneHotVector)+1):
    for combination in itertools.combinations(entitiesOneHotVector, r):
        combination_key = np.array(list(combination)).tostring()
        print(combination_key in ngrams["whereAtrrsDict"])
        if  combination_key in ngrams["whereAtrrsDict"] and len(list(combination)) > len_best_combination:
            print("Hello")
            best_combination = combination
            best_combination_key = combination_key
            len_best_combination = len(list(combination))

In [ ]:
best_combination

In [ ]:
#####get max number of entities in dataset and get total number of unique attributes ###
import json
datasets = ["cosql","greatSql","nvBench","sparc","splash","topics"]
maxEntities = 0
totalSelectAttrs = 0
totalWhereAttrs = 0
d = ""
l = {}
for dataset in datasets:
    with open('/home/nada/GP/GP/GP/notebooks/preparingDatasets/finalOutputs/'+dataset+'.json', 'r',encoding='UTF-8') as file:
        print(dataset)
        data = json.load(file)
        for query in data:
            for q in query["allQueries"]:
                totalSelectAttrs = totalSelectAttrs + len(list(set(q["selectAttrs"])))
                totalWhereAttrs = totalWhereAttrs + len(list(set(q["whereAttrs"])))
                if len(q["entities"]) > maxEntities:
                    maxEntities = len(q["entities"])
                    l = q


print(maxEntities)
print(totalSelectAttrs)
print(totalWhereAttrs)
print(l)

In [ ]:
# 1-gram => prob of selecting attribute in general
# 2-gram => prob of selecting attribute given entity
# 3-gram => prob of selecting attribute given 2 entities 
# and so on .....
# max number of entities 12 => 13-gram 


In [3]:
#####get unigram (calculating frequencies) for attributes ###
import json
datasets = ["cosql","greatSql","nvBench","sparc","splash","academic","advising","geography","imdb","restaurants","scholar","spider","yelp"]
ngramsDict = {}
ngramsDict["selectAttrsDict"] = {}
ngramsDict["whereAtrrsDict"] = {}
for dataset in datasets:
    with open('/home/nada/GP/GP/notebooks/preparingDatasets/finalOutputs/'+dataset+'.json', 'r',encoding='UTF-8') as file:
        print(dataset)
        data = json.load(file)
        for query in data:
            if len(query["allQueries"]) > 1:
                continue    
            for q in query["allQueries"]:
                for attr in list(set(q["selectAttrs"])):
                    if attr.find(".") != -1:
                        attr = attr.split(".")[1]
                    attr = attr.split("_")
                    attr = [get_lemma(word) for word in attr]
                    attrOneHotVector = (getKeyWordsVector(attr)).tostring()
                    if ngramsDict["selectAttrsDict"].get(attrOneHotVector) is None:
                        ngramsDict["selectAttrsDict"][attrOneHotVector] = 1
                    else:
                        ngramsDict["selectAttrsDict"][attrOneHotVector] += 1
                if dataset=="greatSql":
                    print(q["whereAttrs"])
                whereAttrs = [[atr[0],atr[2]] if atr[2]!="value" else [atr[0]]  for atr in q["whereAttrs"]]
                whereAttrs = flattenList(whereAttrs)
                for attr in list(set(whereAttrs)):
                    if attr.find(".") != -1:
                        attr = attr.split(".")[1]
                    attr = attr.split("_")
                    attr = [get_lemma(word) for word in attr]
                    attrOneHotVector = (getKeyWordsVector(attr)).tostring()
                    if ngramsDict["whereAtrrsDict"].get(attrOneHotVector) is None:
                        ngramsDict["whereAtrrsDict"][attrOneHotVector] = 1
                    else:
                        ngramsDict["whereAtrrsDict"][attrOneHotVector] += 1


cosql


/home/nada/.local/lib/python3.7/site-packages/ipykernel_launcher.py:20: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
/home/nada/.local/lib/python3.7/site-packages/ipykernel_launcher.py:34: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.


greatSql
[['account.id', '>=', 'value', 'None']]
[['answer.body', 'like', 'value', 'None']]
[['tape.id', '>=', 'value', 'None']]
[['event.id', '=', 'value', 'None']]
[['author.id', '>=', 'value', 'None']]
[['member.last_name', '=', 'value', 'None']]
[]
[['tag.id', 'in', 'value', 'None']]
[['order.date', '=', 'value', 'None']]
[['broker.id', '!=', 'value', 'None']]
[['member.id', '>=', 'value', 'None']]
[]
[]
[['teacher.first_name', '=', 'value', 'None']]
[]
[['room.id', '<=', 'value', 'None']]
[]
[]
[['product.price', '<', 'value', 'None']]
[['policy.id', '<=', 'value', 'None']]
[['image.id', '!=', 'value', 'None']]
[]
[['material.id', '!=', 'value', 'None']]
[['teacher.last_name', '=', 'value', 'None']]
[]
[]
[['event.id', '>=', 'value', 'None']]
[]
[]
[]
[]
[['account.id', '>=', 'value', 'None']]
[]
[['channel.id', '<', 'value', 'None']]
[]
[['claim.id', '>=', 'value', 'None']]
[['association_type.code', 'not like', 'value', 'None']]
[['tape_category.code', 'like', 'value', 'None']]


In [4]:
import pickle

with open("/home/nada/GP/GP/src/SearchEngine/nGrams/unigram.pickle", 'wb') as handle:
    pickle.dump(ngramsDict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [5]:
def createNgramsDict(attributes,name,combinations):
    for combination in combinations:
        for attr in list(set(attributes)):
            if attr.find(".") != -1:
                attr = attr.split(".")[1]
            attr = attr.split("_")
            attrOneHotVector = (getKeyWordsVector(attr)).tostring()
            if nGramsDict[name][np.array(list(combination)).tostring()].get(attrOneHotVector) is None:
                nGramsDict[name][np.array(list(combination)).tostring()][attrOneHotVector] = 0
            nGramsDict[name][np.array(list(combination)).tostring()][attrOneHotVector] += 1

In [6]:
import json
import itertools
import numpy as np
import pickle
nGramsDict = {}
nGramsDict["selectAttrsDict"] = {}
nGramsDict["whereAtrrsDict"] = {}
datasets = ["cosql","greatSql","nvBench","sparc","splash","academic","advising","geography","imdb","restaurants","scholar","spider","yelp"]
entitiesProp={}
for dataset in datasets:

    with open('/home/nada/GP/GP/notebooks/preparingDatasets/finalOutputs/'+dataset+'.json', 'r',encoding='UTF-8') as file:
        print(dataset)
        data = json.load(file)
        for query in data:
            if len(query["allQueries"]) > 1:
                continue   
            for q in query["allQueries"]:
                entitiesOneHotVector = []
                combinations = []
                for entity in list(set(q["entities"])):
                    entity = entity.split("_")
                    entityOneHotVector = (getKeyWordsVector(entity)).tostring()
                    if entitiesProp.get(entityOneHotVector) is None:
                        entitiesProp[entityOneHotVector] = 0
                    entitiesProp[entityOneHotVector] += 1
                    entitiesOneHotVector.append(entityOneHotVector)
                for r in range(len(entitiesOneHotVector)+1):
                    for combination in itertools.combinations(entitiesOneHotVector, r):
                        if len(list(combination)) > 0:
                            combinations.append(list(combination))
                            nGramsDict["selectAttrsDict"][np.array(list(combination)).tostring()] = {}
                            nGramsDict["whereAtrrsDict"][np.array(list(combination)).tostring()] = {}

                whereAttributes = [[atr[0],atr[2]] if atr[2]!="value" else [atr[0]]  for atr in q["whereAttrs"]]
                whereAttributes = flattenList(whereAttributes)
                
                createNgramsDict(q["selectAttrs"],"selectAttrsDict",combinations)
                createNgramsDict(whereAttributes,"whereAtrrsDict",combinations)
                        


cosql


/home/nada/.local/lib/python3.7/site-packages/ipykernel_launcher.py:23: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
/home/nada/.local/lib/python3.7/site-packages/ipykernel_launcher.py:32: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
/home/nada/.local/lib/python3.7/site-packages/ipykernel_launcher.py:33: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
/home/nada/.local/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  import sys
/home/nada/.local/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  
/home/nada/.local/lib/python3.7/site-packages/ipykernel_launcher.py:9: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  if __name__ == "__main__":
/home/nada/.local/lib/python3.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: tostring() is deprec

greatSql
nvBench
sparc
splash
academic
advising
geography
imdb
restaurants
scholar
spider
yelp


In [ ]:
entity = "award_player"
entity = entity.split("_")
entitiesOneHotVector = []
entityOneHotVector = (getKeyWordsVector(entity)).tostring()
keys = (list(nGramsDict["selectAttrsDict"].keys()))[:10]
print(nGramsDict["selectAttrsDict"][entityOneHotVector].values())

    

In [ ]:
entity = "event_participant"
entity = entity.split("_")
entitiesOneHotVector = []
entityOneHotVector = (getKeyWordsVector(entity)).tostring()
vv = getKeyWordsVector(entity)
for i in vv:
    print(i)
entitiesOneHotVector.append(entityOneHotVector)
for r in range(len(entitiesOneHotVector)+1):
    for combination in itertools.combinations(entitiesOneHotVector, r):
        if len(list(combination)) > 0:
            if nGramsDict["whereAtrrsDict"].get(np.array(list(combination)).tostring()):
                print("hello")

In [7]:
import pickle
with open("/home/nada/GP/GP/src/SearchEngine/nGrams/ngrams.pickle", 'wb') as handle:
    pickle.dump(nGramsDict, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [10]:
nGramsDict={}

In [8]:
entity = "ticket"
entity = entity.split("_")
entityOneHotVector = (getKeyWordsVector(entity)).tostring()
entitiesProp[entityOneHotVector]

/home/nada/.local/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  This is separate from the ipykernel package so we can avoid doing imports until


366

In [9]:
import pickle
with open("/home/nada/GP/GP/src/SearchEngine/nGrams/entities.pickle", 'wb') as handle:
    pickle.dump(entitiesProp, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [ ]:
import pickle
ngramFile = open("/home/nada/GP/GP/GP/src/SearchEngine/nGrams/ngrams.pickle",'rb')
nGramsDict = pickle.load(ngramFile)